In [2]:
import os
os.chdir(os.getcwd())

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from openpyxl import load_workbook

In [4]:
import xlsxwriter
from scipy.signal import argrelextrema

In [5]:
# mean absolute deviation function
def mad(data, axis=None):
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)

In [6]:
from scipy.optimize import curve_fit

In [7]:
def func(x, a, b):
    return a * x + b

In [8]:
speed=[10,30,50,100,150,200,300,400,600,800,1000,1400,1800]
weights=[41,53,65,77,89,136,184,232]
headers=('static_strain(mm)','static_force','min_mean','min_del','min_del_mad','max_mean','max_del','max_del_mad','')

df= pd.DataFrame(columns=headers,index=weights)

headers2=['static_force_a','static_force_b','static_strain_a','static_strain_b','kinetic_max_mean_a','kinetic_max_mean_b','kinetic_min_mean_a','kinetic_min_mean_b','kinetic_del_avg','kintetic_del_mad','mad/avg']
df2= pd.DataFrame(columns=headers2,index=speed)


In [7]:
writer = pd.ExcelWriter('analysis.xlsx', engine='xlsxwriter')

for i,s in enumerate(speed):
    for w in weights:
        file='friction coeficient_'+str(w)+'g.xls'
        dfs = pd.read_excel(file, sheet_name=i+3)
        
        #clean up columns
        dfs.columns=['time','strain','force','work']
        dfs=dfs.drop(dfs.index[[0,1]])
        
        max=dfs['force'].max()
        df[headers[0]][w]=dfs[dfs['force']==max].strain.values[0]
        df[headers[1]][w]=max
        
        n=9 # the bigger the more comparison width
        dfs['min']=dfs.iloc[argrelextrema(dfs.force.values, np.less_equal, order=n)[0],2]
        dfs['max']=dfs.iloc[argrelextrema(dfs.force.values, np.greater_equal, order=n)[0],2]

        #removing the NaNs
        dfs2=dfs.dropna(subset=['min'])
        dfs3=dfs.dropna(subset=['max'])

        #calculating the delta
        pd.options.mode.chained_assignment = None  # default='warn'
        dfs2['del']=dfs2.strain-dfs2.strain.shift(1)
        dfs3['del']=dfs3.strain-dfs3.strain.shift(1)

        #removing the too close points
        dfs2=dfs2[dfs2['del']>0.5*dfs2['del'].mean()]
        dfs3=dfs3[dfs3['del']>0.5*dfs3['del'].mean()]

        dfs2=dfs2[dfs2['del']<1.9*dfs2['del'].mean()]
        dfs3=dfs3[dfs3['del']<1.9*dfs3['del'].mean()]

        #removing NaNs
        dfs2=dfs2.dropna(subset=['del'])
        dfs3=dfs3.dropna(subset=['del'])

        #calculating mean frequency and mean absolute deviation
        #print(dfs2['min'].mean(),',',dfs2['del'].mean(),',',mad(dfs2['del'].values), end='')
        df[headers[2]][w]=dfs2['min'].mean()
        df[headers[3]][w]=dfs2['del'].mean()
        df[headers[4]][w]=mad(dfs2['del'].values)
        #print(dfs3['max'].mean(),',',dfs3['del'].mean(),',',mad(dfs3['del'].values))
        df[headers[5]][w]=dfs3['max'].mean()
        df[headers[6]][w]=dfs3['del'].mean()
        df[headers[7]][w]=mad(dfs3['del'].values)
        
    df.to_excel(writer, sheet_name=str(s))
    popt, pcov = curve_fit(func, weights, df[headers[0]].values.astype(float))
    df2[headers2[2]][s]=popt[0]
    df2[headers2[3]][s]=popt[1]

    popt, pcov = curve_fit(func, weights, df[headers[1]].values.astype(float))
    df2[headers2[0]][s]=popt[0]*1000/9.81
    df2[headers2[1]][s]=popt[1]

    popt, pcov = curve_fit(func, weights, df[headers[5]].values.astype(float))
    df2[headers2[4]][s]=popt[0]*1000/9.81
    df2[headers2[5]][s]=popt[1]

    popt, pcov = curve_fit(func, weights, df[headers[2]].values.astype(float))
    df2[headers2[6]][s]=popt[0]*1000/9.81
    df2[headers2[7]][s]=popt[1]

    df2[headers2[8]][s]=(df[headers[3]].mean()+df[headers[6]].mean())/2
    df2[headers2[9]][s]=(df[headers[4]].mean()+df[headers[7]].mean())/2

df2[headers2[10]]=df2[headers2[9]]/df2[headers2[8]]
df2.to_excel(writer, sheet_name='analysis')
writer.save()

In [ ]:
%matplotlib inline
# %matplotlib qt

file='friction coeficient_'+str(weights[0])+'g.xls'
dfs = pd.read_excel(file, sheet_name=0+3)

plt.plot(dfs['strain'], dfs['force'],label='data')
plt.scatter(dfs2['strain'], dfs2['min'],label='min')
plt.scatter(dfs3['strain'], dfs3['max'],label='max')
plt.show()

In [157]:
#%matplotlib inline
%matplotlib qt
plt.plot(df2.index, df2[headers2[0]],label='data')
plt.show()

In [49]:
#%matplotlib inline
%matplotlib qt
file2='analysis.xlsx'
df3 = pd.read_excel(file2, sheet_name='analysis')
h=0
plt.plot(df3.index, df3[headers2[h]],label=headers2[h])
    #plt.scatter(df3[headers2[10]], df3[headers2[10]],label=str(a))

plt.show()

In [2]:
from interpacf import interpolated_acf, dominant_period

In [8]:
# writer = pd.ExcelWriter('analysis_2.xlsx', engine='xlsxwriter')

df4=pd.DataFrame(columns=weights,index=speed)

for i,s in enumerate(speed):
    for w in weights:
        file='friction coeficient_'+str(w)+'g.xls'
        dfs = pd.read_excel(file, sheet_name=i+3)
        
        #clean up columns
        dfs.columns=['time','strain','force','work']
        dfs=dfs.drop(dfs.index[[0,1]])
        
        dfs.sort_values(by=['strain'],inplace=True)
        
        flux = dfs['force'].values.astype(float) - np.mean(dfs['force'].values.astype(float))

        lag, acf = interpolated_acf(dfs['strain'].values.astype(float), flux)
        period = dominant_period(lag, acf, fwhm=1, plot=False)
        
        df4[w][s]=period

df4

c:\users\fsfar\appdata\local\programs\python\python37-32\lib\site-packages\interpacf-0.1-py3.7.egg\interpacf\interpacf.py:118: NonzeroMedianWarning: Have you normalized your fluxes so that their median is near zero?


,41,53,65,77,89,136,184,232
10,1.24616,1.18799,1.2234,1.43279,1.14391,1.68699,2.16073,1.90833
30,0.937256,0.821824,0.837429,1.02227,0.668449,1.22482,1.39209,1.35946
50,0.777338,0.826931,0.79303,0.876446,0.77736,1.07507,1.17411,1.17428
100,0.666094,0.699399,0.699399,0.799318,0.765997,0.932531,0.965822,1.06575
150,0.650883,0.89724,0.747671,0.751047,0.801117,0.851154,0.95129,1.0014
200,0.666094,0.732693,0.730217,0.799315,0.865959,0.999184,1.06575,1.0025
300,0.699402,0.799316,0.799316,0.799316,0.899231,0.999146,1.10154,1.09906
400,12.2563,0.932545,0.799988,1.06577,1.06577,1.19898,1.19899,1.19897
600,7.19385,7.60206,16.0045,1.40038,1.3988,1.60044,1.59863,1.59863
800,12.5333,14.4121,12.277,9.0666,13.3446,12.8,13.3333,61.8666


In [11]:
%matplotlib qt
df4.plot()

In [6]:
def func_2(x,a,b):
    return a*x**(2.0/3)+b*x

In [19]:
df6=pd.DataFrame(columns=['a','b'],index=speed)
for i,s in enumerate(speed):
    df5=pd.read_excel('analysis.xlsx', sheet_name=str(s))
    popt, pcov = curve_fit(func_2, df5.index.values.astype(float)*9.81/1000, df5['max_mean'].values.astype(float))
    df6['a'][s]=popt[0]
    df6['b'][s]=popt[1]
df6

,a,b
10,0.177009,0.149101
30,0.172082,0.117917
50,0.249109,0.0201777
100,0.187605,0.0628121
150,0.238538,0.001697
200,0.238581,-0.000790347
300,0.200946,0.0190311
400,0.21915,-0.00045655
600,0.13747,0.0754179
800,0.188483,-0.00474815


In [9]:
%matplotlib qt

writer2 = pd.ExcelWriter('analysis_2.xlsx', engine='xlsxwriter')

for w in weights:
    df7=pd.DataFrame(columns=['kin_max'],index=speed)
    for i,s in enumerate(speed):
        df8=pd.read_excel('analysis.xlsx', sheet_name=str(s))
        df7['kin_max'][s]=df8['max_mean'][w]
    df7.to_excel(writer2, sheet_name=str(w))
        
        
writer2.save()